# A workflow to find outlier expansions

## Authors in alphabetical order

- [Madeline Couse, SickKids](mailto:madeline.couse@sickkids.ca)
- [Giulia Del Gobbo, CHEO](mailto:gdelgobbo@cheo.on.ca)
- [Egor Dolzhenko, PacBio](mailto:edolzhenko@pacificbiosciences.com)
- [Adam English, BCM](mailto:adam.english@bcm.edu)
- [Tom Mokveld, PacBio](mailto:tmokveld@pacificbiosciences.com)

## Introduction

Suppose that you want to look for repeat expansions in one or more of your case samples relative to a set of controls. Let's assume that you have already aligned the reads, ran TRGT, and merged the results into an allele database file `alleles.gz` that has three columns: repeat id, sample id, and allele sequences. Let's use the next code block to set the path to your allele database and also define ids of case samples.  

In [1]:
from pathlib import Path

alleles_path = Path("input/alleles.gz").resolve(strict=True)

case_ids = ["HG00280", "HG00438", "HG00621"] # Assume that all other samples are controls

Now let's confirm that allele database file has the expected structure:

In [2]:
! zcat < {alleles_path} | head -n 3

chr9_100000411_100000479_A	HG00099	AGTGATAAAGCAAGATCCTGTATCAAAAAAAAAAAAAAAAAAACTTTCTCCCTTCTTCAATTTTTTGG,AGTGATAAAGCAAGATCCTGTATCAAAAAAAAAAAAAAAAAACTTTCTCCCTTCTTCAATTTTTTGG
chr9_100000411_100000479_A	HG00280	AGTGATAAAGCAAGATCCTGTATCAAAAAAAAAAAAAAAAAAACTTTCTCCCTTCTTCAATTTTTTGG,AGTGATAAAGCAAGATCCTGTATCAAAAAAAAAAAAAAAAAACTTTCTCCCTTCTTCAATTTTTTGG
chr9_100000411_100000479_A	HG00323	AGTGATAAAGCAAGATCCTGTATCAAAAAAAAAAAAAAAAAACTTTCTCCCTTCTTCAATTTTTTGG,AGTGATAAAGCAAGATCCTGTATCAAAAAAAAAAAAAAAAAAAACTTTCTCCCTTCTTCAATTTTTTGG


Let's also create directories for temporary files and the final output:

In [3]:
scratch_path = Path("scratch").mkdir(exist_ok=True)
output_path = Path("output").mkdir(exist_ok=True)

## Find outliers

Let's create some code for extracting alleles from the database file.

In [4]:
import gzip
import itertools
import numpy as np
from collections import namedtuple

RepeatRec = namedtuple("RepeatRec", "sample short_allele long_allele")

def get_repeat_recs(path):
    def parse_alleles(group):
        alleles = list(line.decode("utf8").split() for line in group)
        alleles = [(rec[1], rec[2].split(",")) for rec in alleles]
        return alleles
    
    with gzip.open(path, "r") as file:
        for trid, group in itertools.groupby(file, key=lambda line: line.decode("utf8").split()[0]):
            alleles = parse_alleles(group)
            repeat_recs = [(s, [len(a) for a in als]) for s, als in alleles]
            repeat_recs = [RepeatRec(s, min(als), max(als)) for s, als in repeat_recs]
            
            yield trid, repeat_recs

The following code with resample quantiles.

In [5]:
def resample_quantiles(counts, num_resamples):
    """Based on https://github.com/Illumina/ExpansionHunterDenovo/blob/master/scripts/core/common.py"""
    resamples = np.random.choice(counts, len(counts) * num_resamples)
    resamples = np.split(resamples, num_resamples)

    resampled_quantiles = []
    for resample in resamples:
        quantile = np.quantile(resample, 0.95)
        resampled_quantiles.append(quantile)

    return resampled_quantiles


def get_counts_with_finite_std(counts):
    if len(set(counts)) == 1:
        return counts[:-1] + [counts[-1] + 0.1]
    return counts


def get_cutoff(quantiles):
    mean = np.mean(quantiles)
    std = max(1, np.std(quantiles))
    cutoff = mean + std
    return cutoff

In [6]:
HitRec = namedtuple("HitRec", "trid sample allele_type allele_len control_range")

def get_hits(allele_type, repeat_recs):
    assert allele_type in ["long", "short"]
    allele_index = 2 if allele_type == "long" else 1
    cases, controls = {}, []
    for rec in repeat_recs:
        if rec.sample in case_ids:
            cases[rec.sample] = rec[allele_index]
        else:
            controls.append(rec[allele_index])
    quantiles = resample_quantiles(controls, 100)
    quantiles = get_counts_with_finite_std(quantiles)
    cutoff = get_cutoff(quantiles)
    
    for case, allele_len in cases.items():
        if allele_len > cutoff:
            yield case, allele_len, (min(controls), max(controls))


hits = []
for trid, repeat_recs in get_repeat_recs(alleles_path):
    for hit_type in ["long", "short"]:
        for case, allele_len, control_range in get_hits(hit_type, repeat_recs):
            hits.append(HitRec(trid, case, hit_type, allele_len, control_range))


print(f"Found {len(hits)} hits")

Found 1751 hits
